In [2]:
import numpy as np
import pandas as pd
import matplotlib
import sklearn
import math
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.svm import SVC
import functools
from copy import deepcopy
import math

In [3]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
np.set_printoptions(precision=4, suppress=True)

In [4]:
x = '..\Problem\shuffled-full-set-hashed.csv'
data = pd.read_csv(x, header=None)
df = pd.DataFrame(data)
df.columns = ['output','input']
df.dropna(inplace=True)

In [5]:
# WordMapper
mapper = {}
ID = 0
for i in range(len(df)):
    for word in df.iloc[i][1].split(' '):
        if word not in mapper:
            mapper[word] = (word, ID)
            ID += 1

In [6]:
# WordSwapperWithMapper
temp = []
x = []
for i in range(len(df)):
    for word in df.iloc[i][1].split(' '):
        temp.append(mapper[word][1])
    df.iloc[i][1] = temp[:]
    temp = []

In [7]:
# Output Dic {Output: [Input(s)]}
docForEach = {}
for i in range(len(df)):
    if df.iloc[i][0] in docForEach:
        docForEach[df.iloc[i][0]].extend(df.iloc[i][1][:])
    else:
        docForEach[df.iloc[i][0]] = df.iloc[i][1][:]
keys = docForEach.keys()
lenForEach = {}        
temp = {}
for key in docForEach:
    lenForEach[key] = len(docForEach[key])
    for word in docForEach[key]:
        if word in temp:
            temp[word] += 1
        else:
            temp[word] = 1
    docForEach[key] = deepcopy(temp)
    temp={}

In [8]:
docForAll = {key: {i: 0 for i in range(len(mapper))} for key in docForEach.keys()}

In [9]:
len(mapper)

1037934

In [10]:
for key in docForAll:
    for word in docForAll[key]:
        if word in docForEach[key]:
            docForAll[key][word] = 1

In [11]:
DataFrame = pd.DataFrame(data=[docForAll[key] for key in keys], index=keys)

In [12]:
DataFrame

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,1037684,1037685,1037686,1037687,1037688,1037689,1037690,1037691,1037692,1037693,1037694,1037695,1037696,1037697,1037698,1037699,1037700,1037701,1037702,1037703,1037704,1037705,1037706,1037707,1037708,1037709,1037710,1037711,1037712,1037713,1037714,1037715,1037716,1037717,1037718,1037719,1037720,1037721,1037722,1037723,1037724,1037725,1037726,1037727,1037728,1037729,1037730,1037731,1037732,1037733,1037734,1037735,1037736,1037737,1037738,1037739,1037740,1037741,1037742,1037743,1037744,1037745,1037746,1037747,1037748,1037749,1037750,1037751,1037752,1037753,1037754,1037755,1037756,1037757,1037758,1037759,1037760,1037761,1037762,1037763,1037764,1037765,1037766,1037767,1037768,1037769,1037770,1037771,1037772,1037773,1037774,1037775,1037776,1037777,1037778,1037779,1037780,1037781,1037782,1037783,1037784,1037785,1037786,1037787,1037788,1037789,1037790,1037791,1037792,1037793,1037794,1037795,1037796,1037797,1037798,1037799,1037800,1037801,1037802,1037803,1037804,1037805,1037806,1037807,1037808,1037809,1037810,1037811,1037812,1037813,1037814,1037815,1037816,1037817,1037818,1037819,1037820,1037821,1037822,1037823,1037824,1037825,1037826,1037827,1037828,1037829,1037830,1037831,1037832,1037833,1037834,1037835,1037836,1037837,1037838,1037839,1037840,1037841,1037842,1037843,1037844,1037845,1037846,1037847,1037848,1037849,1037850,1037851,1037852,1037853,1037854,1037855,1037856,1037857,1037858,1037859,1037860,1037861,1037862,1037863,1037864,1037865,1037866,1037867,1037868,1037869,1037870,1037871,1037872,1037873,1037874,1037875,1037876,1037877,1037878,1037879,1037880,1037881,1037882,1037883,1037884,1037885,1037886,1037887,1037888,1037889,1037890,1037891,1037892,1037893,1037894,1037895,1037896,1037897,1037898,1037899,1037900,1037901,1037902,1037903,1037904,1037905,1037906,1037907,1037908,1037909,1037910,1037911,1037912,1037913,1037914,1037915,1037916,1037917,1037918,1037919,1037920,1037921,1037922,1037923,1037924,1037925,1037926,1037927,1037928,1037929,1037930,1037931,1037932,1037933
DELETION OF INTEREST,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
RETURNED CHECK,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,1

In [13]:
def computeTF(wordDict, length):
    return {word: round(count/length, 5) for word,count in wordDict.items()}

In [14]:
tfForEach = {key: computeTF(docForEach[key],lenForEach[key]) for key in docForEach.keys()}

In [15]:
def computeIDF(docList):
    idfDict = {}
    N = len(docList)
    
    #counts the number of documents that contains word w
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] +=1
                
    #divide N by denominator above, take the log of that
    for word, val in idfDict.items():
        idfDict[word]= math.log(N / float(val)) 

    return idfDict

In [16]:
idfForAllWords = computeIDF([docForAll[key] for key in keys])

In [17]:
def computeTFIDF(tf, idfs):
    tfidf = {}
    for word, val in tf.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [18]:
tfidfForEach = {key: computeTFIDF(tfForEach[key], idfForAllWords) for key in keys}

In [19]:
DataFrameTFIDF = pd.DataFrame(data=[tfidfForEach[key] for key in keys], index=keys)

In [20]:
DataFrameTFIDF

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,978571,978572,978573,978574,978575,978576,978578,978579,978580,978581,978582,978583,978584,987091,987092,987093,987094,987095,987096,987097,987098,987099,987100,987101,987102,987103,987104,987105,987106,987107,987108,987109,987110,987111,987112,987113,987114,987115,987116,987117,987118,987119,987120,987121,987122,987123,987124,987125,987126,987127,987128,987129,987130,987131,987132,987133,987134,987135,987136,987137,987138,987139,987140,987141,987142,987143,987144,987145,987146,987147,987148,987149,987150,987151,987152,987153,987154,987155,987156,987157,987158,987159,987160,987161,987162,987163,987164,987165,987166,987167,987168,987169,987170,987171,987172,987173,987174,987175,987176,987177,987178,987179,987180,987181,987182,987183,987184,987185,987186,987187,987188,987189,987190,987191,987192,987193,987194,987195,987196,987197,987198,987199,987200,987201,987202,987203,987204,987205,987206,987207,987208,987209,992730,992731,992732,992733,992734,992735,992736,992737,992738,992739,993986,993987,1012530,1012531,1012532,1012533,1012534,1012535,1012536,1012537,1012538,1012539,1012540,1012541,1012542,1012543,1012544,1012545,1012546,1012547,1012548,1012549,1012550,1012551,1012552,1012553,1012554,1012555,1012556,1012557,1012558,1012559,1012560,1012561,1012562,1012563,1012564,1012565,1012566,1012567,1012568,1012569,1012570,1012571,1012572,1012573,1012574,1012575,1012576,1012577,1012578,1017073,1017074,1017075,1017076,1017077,1017078,1017079,1017080,1017081,1017429,1017430,1017431,1017432,1017433,1017434,1017435,1017436,1017437,1017438,1025362,1025363,1025364,1025365,1025366,1025367,1025368,1025369,1025370,1025371,1025372,1025373,1025374,1025375,1025376,1025377,1025378,1025379,1025380,1025381,1027911,1027912,1027913,1027914,1033530,1034711,1034712,1034713,1034714,1034715,1034716,1034717,1034718,1034719,1034720,1034721,1034722,1036698
DELETION OF INTEREST,0.0,0.000101,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000,0.000018,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000e+00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000001,0.0,0.0,0.00000,0.000000,0.0,0.0,7.410797e-07,0.000000,7.410797e-07,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000002,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.000040,0.0,0.0,0.0,0.0,0.000060,0.0,0.0,0.000119,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000023,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000002,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.223239e-06,0.0,4.446478e-06,0.0,0.0,0.000000,2.964319e-06,0.0,0.0,0.000003,0.000006,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,2.964319e-06,0.000014,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000001,0.000005,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,7.410797e-07,0.0,0.000000,0.00000,0.000000e+00,0.0,0.000000e+00,0.0,0.0,0.000000,0.0,0.0,0.0,0.000084,0.000263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000044,0.000139,0.0,0.000083,0.0,2.9643

In [21]:
DataFrameTFIDF.dropna(axis=1, how='all',inplace=True)
DataFrameTFIDF.drop(DataFrameTFIDF.columns[DataFrameTFIDF.iloc[-1,:]==0], axis=1,inplace=True)

In [22]:
rareWords = {key: {} for key in keys}
for key in keys:
    rareWords[key] = dict.fromkeys(
        map(
            lambda x: x[0],
            sorted(
                filter(lambda x: x[1] != 0, DataFrameTFIDF.loc[key].dropna().items()),
                key=lambda x: x[1],
                reverse=True
            )[:100]
        ),
        None
    )

In [29]:
rareWords = {
    key: dict.fromkeys(
        map(
            lambda x: x[0],
            sorted(
                filter(lambda x: x[1] != 0, DataFrameTFIDF.loc[key].dropna().items()),
                key=lambda x: x[1],
                reverse=True
            )[:100]
        ),
        None
    ) for key in keys
}


In [30]:
rareWords['BILL']

{5841: None,
 757: None,
 1700: None,
 700: None,
 1922: None,
 688: None,
 246: None,
 30359: None,
 24475: None,
 25152: None,
 10488: None,
 21957: None,
 30358: None,
 7593: None,
 7605: None,
 1706: None,
 249: None,
 3810: None,
 12347: None,
 1966: None,
 7109: None,
 1003: None,
 1948: None,
 12745: None,
 30355: None,
 30499: None,
 729: None,
 12706: None,
 3855: None,
 1730: None,
 3808: None,
 1595: None,
 1718: None,
 1916: None,
 1963: None,
 15030: None,
 3822: None,
 1753: None,
 1702: None,
 1405: None,
 3805: None,
 5834: None,
 6684: None,
 7118: None,
 1822: None,
 3798: None,
 1731: None,
 9069: None,
 26376: None,
 499: None,
 5283: None,
 3577: None,
 1784: None,
 7591: None,
 15033: None,
 737: None,
 3823: None,
 859: None,
 6439: None,
 7559: None,
 5056: None,
 1593: None,
 21665: None,
 18184: None,
 1486: None,
 930: None,
 3793: None,
 245: None,
 1406: None,
 1607: None,
 3817: None,
 3800: None,
 6991: None,
 10519: None,
 10529: None,
 11340: None,
 292

In [24]:
# sorted(, key=lambda x: x[1], reverse=True)
x = list(filter(lambda x: x[1] != float('nan') or x[1] != 0, DataFrameTFIDF.loc[key].items()))

In [25]:
# rareWords['BILL']

In [26]:
# x = list(DataFrameTFIDF.loc['BILL'].dropna().items())
# for i in x:
#     print(i[1] == 0, i)
# DataFrameTFIDF.loc['BILL'].dropna().items()

In [27]:
# Testing
tester = 'afb1e3806fc1 de9738ee8b24 fb2cd24a447a ed5d3a65ee2d fa8e47e29a57 422068f04236 6ce6cc5a3203 e8dfec3ed7ae 56a03770d398 21e314d3afcc 9bc65adc033c 01a21c4d7fcc e7e059c82399 4ad52689d690 3486e5fe0d73 c337a85b8ef9 422068f04236 094453b4e4ae 48d657cd9861 ad4440ac97a5 56a03770d398 21e314d3afcc 0cc20df880e1 55c2d4d8f1de ff1c26ea0b6f ee917382bd21 ea51fa83c91c 73801426ea65 cae1e5c5d33a c9a53ea6e219 e8dfec3ed7ae f7ae6f8257da 4ad52689d690 586242498a88 d944e076fa92 9bbe6d06c8d4 b9699ce57810 01a21c4d7fcc 56a03770d398 21e314d3afcc cbd6e2e7a804 d38820625542 2ee1e0de2738 04503bc22789 7d9e333a86da f96708dc5a81 df3545376d58 e22c71895f3d 6b8e946876ae 6fb9578fdedb df3545376d58 0f88ca127938 6a71a81e16e3 cc9e05bc2a86 f751d3d92e0f 947d07cce521 f0e8ae066e0f b291eb0bb9e7 104b036e83f9 cf10af44938e 2903ddf3d7c0 94a8b0893dff eb60ef77d773 06c0da3d5055 4ad52689d690 3486e5fe0d73 c337a85b8ef9 04503bc22789 1d4249bb404a 5c02c2aaa67b 0d7e91148dc6 b9699ce57810 73801426ea65 25c57acdf805 b40d7fa0772c e7e059c82399 2902ad7bd2bb 1b8d4107c33e 557ec6c63cf9 fff6d974e9ed be619ffab098 90a6ff829c50 6bc963b8a2b5 87a211b76bd3 8df4106b10cb 90a6ff829c50 f96708dc5a81 df3545376d58 a11ab7a9914c 6fb9578fdedb 8d6768212702 cc9e05bc2a86 947d07cce521 b291eb0bb9e7 ea51fa83c91c a149ed2d9b56 73801426ea65 ff1c26ea0b6f 6d1fb90988cf'

# WordSwapperWithMapper
tester = set(map(lambda x: mapper[x][1], tester.split(' ')))
# print(tester)

# CommonWordCheck
hitCount = {key:0 for key in keys}
for key in keys:
    for i in tester:
        if i in rareWords[key]:
            hitCount[key] += 1
print(max(hitCount.items(), key=lambda x: x[1])[0])

DELETION OF INTEREST
